In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Load the saved model
model = tf.keras.models.load_model('action.h5')



In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [3]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [4]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
def extract_keypoints(results):
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [7]:

# 30 videos worth of data
no_sequences = 30

# videos are going to be 30 frames in length
sequence_length = 30

# Test in Real Time

In [8]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16), (245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16)]

actions = np.array(['pinch-fingers spread','multiple-select','box-select', 'scale-uniform', 'scale-X-axis', 'swipe left-to-right', 'pause', 'delete', 'copy', 'cut'])

# Ensure colors list matches the length of actions list
colors = colors[:len(actions)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        if num < len(actions):  # Ensure num is within bounds
            cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0), 1, cv2.LINE_AA)
    return output_frame


        



In [9]:
import cv2
import numpy as np
import mediapipe as mp

# Load the holistic model and other necessary setups from MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.4

# Load the model (assuming it's already loaded in the environment)
# model = your_loaded_model

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process the frame with MediaPipe Holistic
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks with the specific styling
        draw_styled_landmarks(image, results)
        
        # Prediction logic with 30 frames sequence
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            action = actions[np.argmax(res)]
            predictions.append(np.argmax(res))

            # Visualization logic
            if np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold:
                if len(sentence) > 0 and actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(action)
                elif len(sentence) == 0:
                    sentence.append(action)

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Visualize the probabilities
                image = prob_viz(res, actions, image, colors)

                # Find bounding box coordinates for landmarks
                hand_landmarks = results.left_hand_landmarks or results.right_hand_landmarks
                if hand_landmarks:
                    x_min = min([lm.x for lm in hand_landmarks.landmark])
                    y_min = min([lm.y for lm in hand_landmarks.landmark])
                    x_max = max([lm.x for lm in hand_landmarks.landmark])
                    y_max = max([lm.y for lm in hand_landmarks.landmark])
                    
                    x_min = int(x_min * image.shape[1])
                    x_max = int(x_max * image.shape[1])
                    y_min = int(y_min * image.shape[0])
                    y_max = int(y_max * image.shape[0])

                    # Draw rectangle and text
                    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 0), 2)
                    cv2.putText(image, action, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Show to screen
        cv2.imshow('Hand Gesture Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()